[View in Colaboratory](https://colab.research.google.com/github/x110/MyFastaiNotebooks/blob/master/lang_model_arxiv.ipynb)

# Sentiment Classification


## Introduction

### Objective
The sentiment classification task consists of predicting the polarity (positive or negative) of a given text. 


###  Dataset
The [large moview view dataset](http://files.fast.ai/data/aclImdb.tgz) contains a collection of 50,000 reviews from IMDB. The datset contains an even number of positive and negative reviews. The authors considered only highly polarized reviews. A negative review has a score $\leq 4$ out of 10, a positive review has a score $\geq 7$ out of 10. Nuteral reviews are not included in this dataset, 
The dataset we are using is the IMDb movie review dataset. "This is a dataset of binary sentiment classification. It contains a set of 25,000 higly polar movie reviews for training, and 25,000 for testing.  

### Solution Strategy
Before we try to classify sentiment, we will try to create a language model; that is, a model that can predict the next word in a sentence. Why? Because our model first needs to understand the structure of English, before we can expect it to recognize positive or negative sentiment. 

## Prepare virtual machine on google colab


1. Connect to google drive
2. Install dependencies

In [56]:
!ls

aclImdb.tgz  aclImdb.tgz.1  aclImdb.tgz.2  data  datalab  drive


In [2]:
## Connect to google drive for storage 
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

gpg: keybox '/tmp/tmp6ykv0zxs/pubring.gpg' created
gpg: /tmp/tmp6ykv0zxs/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [3]:
#Install dependencies
!pip install https://github.com/fastai/fastai/archive/master.zip
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision
!pip install kaggle

    100% |████████████████████████████████| 90.0MB 801kB/s 
  Running setup.py bdist_wheel for fastai ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-oo0josb2/wheels/64/3d/9f/d12a217aa2531321c5b9ae96288fcae2687d3b744376e8f94f


Successfully built fastai
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
Hit:1 http://security.ubuntu.com/ubuntu artful-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu artful InRelease
Hit:3 http://ppa.launchpad.net/alessandro-strada/ppa/ubuntu artful InRelease
Hit:4 http://archive.ubuntu.com/ubuntu artful-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu artful-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
4 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state informat

In [4]:
#install More dependencies
!pip install Pillow==4.0.0
!pip install PIL
!pip install image
!apt-get install p7zip-full

  Using cached https://files.pythonhosted.org/packages/37/e8/b3fbf87b0188d22246678f8cd61e23e31caa1769ebc06f1664e2e5fe8a17/Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
  Found existing installation: Pillow 5.1.0
    Uninstalling Pillow-5.1.0:
      Successfully uninstalled Pillow-5.1.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-4).
0 upgraded, 0 newly installed, 0 to remove an

In [5]:
!mkdir .kaggle
!cp drive/kaggle.json /content/.kaggle/
!chmod 600 /content/.kaggle/kaggle.json

mkdir: cannot create directory ‘.kaggle’: File exists


In [6]:
!pip install spacy

torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.


## Downloads

### Download the weights of the trained language model

In [0]:
#already in drive

### Download data

In [9]:
!wget http://files.fast.ai/data/aclImdb.tgz

--2018-06-30 06:47:08--  http://files.fast.ai/data/aclImdb.tgz
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145982645 (139M) [text/plain]
Saving to: ‘aclImdb.tgz.1’

aclImdb.tgz.1       100%[===================>] 139.22M  91.9MB/s    in 1.5s    

2018-06-30 06:47:10 (91.9 MB/s) - ‘aclImdb.tgz.1’ saved [145982645/145982645]



In [0]:
!mkdir data

In [0]:
!tar -xzf aclImdb.tgz	

In [0]:
!mv aclImdb	data/

# Run model under 10 min

## Imports

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

## Language modeling
### Data

In [20]:
!ls

aclImdb.tgz  aclImdb.tgz.1  aclImdb.tgz.2  data  datalab  drive


In [21]:
PATH = 'data/aclImdb/'
TRN_PATH = 'train/all/'
VAL_PATH = 'test/all/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

imdbEr.txt  imdb.vocab  README  test/  train/


In [22]:
trn_files = !ls {TRN}
trn_files[:10]

['0_0.txt',
 '0_3.txt',
 '0_9.txt',
 '10000_0.txt',
 '10000_4.txt',
 '10000_8.txt',
 '1000_0.txt',
 '10001_0.txt',
 '10001_10.txt',
 '10001_4.txt']

In [23]:
review=!cat {TRN}{trn_files[6]}
review[0] 

"I have to say when a name like Zombiegeddon and an atom bomb on the front cover I was expecting a flat out chop-socky fung-ku, but what I got instead was a comedy. So, it wasn't quite was I was expecting, but I really liked it anyway! The best scene ever was the main cop dude pulling those kids over and pulling a Bad Lieutenant on them!! I was laughing my ass off. I mean, the cops were just so bad! And when I say bad, I mean The Shield Vic Macky bad. But unlike that show I was laughing when they shot people and smoked dope.<br /><br />Felissa Rose...man, oh man. What can you say about that hottie. She was great and put those other actresses to shame. She should work more often!!!!! I also really liked the fight scene outside of the building. That was done really well. Lots of fighting and people getting their heads banged up. FUN! Last, but not least Joe Estevez and William Smith were great as the...well, I wasn't sure what they were, but they seemed to be having fun and throwing out 

In [24]:
!find {TRN} -name "*.txt" | xargs cat| wc -w

17486581


In [25]:
!find {VAL} -name "*.txt" | xargs cat | wc -w

5686719


In [26]:
!python -m spacy download en

    100% |████████████████████████████████| 37.4MB 40.6MB/s 
  Running setup.py install for en-core-web-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
spacy_tok=spacy.load('en')

In [28]:
spacy_tok

In [29]:
' '.join(str(v) for v in spacy_tok(review[0]))

"I have to say when a name like Zombiegeddon and an atom bomb on the front cover I was expecting a flat out chop - socky fung - ku , but what I got instead was a comedy . So , it was n't quite was I was expecting , but I really liked it anyway ! The best scene ever was the main cop dude pulling those kids over and pulling a Bad Lieutenant on them ! ! I was laughing my ass off . I mean , the cops were just so bad ! And when I say bad , I mean The Shield Vic Macky bad . But unlike that show I was laughing when they shot people and smoked dope.<br /><br />Felissa Rose ... man , oh man . What can you say about that hottie . She was great and put those other actresses to shame . She should work more often ! ! ! ! ! I also really liked the fight scene outside of the building . That was done really well . Lots of fighting and people getting their heads banged up . FUN ! Last , but not least Joe Estevez and William Smith were great as the ... well , I was n't sure what they were , but they see

In [0]:
TEXT=data.Field(lower=True,tokenize='spacy')#spacy_tok)#https://github.com/fastai/fastai/issues/213

In [31]:
type(list(spacy_tok(review[0]))[0])

spacy.tokens.token.Token

In [0]:
bs=64;bptt=70

In [0]:
FILES=dict(train=TRN_PATH,validation=VAL_PATH,test=VAL_PATH)

In [37]:
%time md = LanguageModelData.from_text_files(PATH,TEXT,**FILES,bs=bs,bptt=bptt,min_freq=10)

CPU times: user 14min 55s, sys: 7.39 s, total: 15min 2s
Wall time: 15min 4s


In [38]:
md.nt

37392

In [39]:
!ls data/aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [41]:
!mkdir {PATH}models

pickle.dump(TEXT,open(f'{PATH}models/TEXT.pkl','wb'))

mkdir: cannot create directory ‘data/aclImdb/models’: File exists


In [42]:
len(md.trn_dl),md.nt,len(md.trn_ds),len(md.trn_ds[0].text)

(4583, 37392, 1, 20540756)

This is the start of the mapping from integer IDs to unique tokens

In [43]:
#'itoi':'string to int'
TEXT.vocab.itos[:12]

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is', 'in', 'it']

In [44]:
TEXT.vocab.stoi['the']

2

Note that in a languageModelData object there is only one item in each dataset: all words of the text joined together

In [45]:
md.trn_ds[0].text[:12]

['i',
 "'ll",
 'keep',
 'this',
 'short',
 'and',
 'sweet',
 ':',
 'anyone',
 'who',
 'found',
 'himself']

torchtext will handle turning these words into integer IDs for us automatically

In [46]:
TEXT.numericalize([md.trn_ds[0].text[0:12]])

Variable containing:
   12
  259
  398
   13
  370
    5
 1096
   94
  272
   43
  268
  331
[torch.cuda.LongTensor of size 12x1 (GPU 0)]

Our language model object will create batches with 64 columns ,and varying sequence lengths of around 70 tokens.

Each batch also contains the exact sane data as labels,but one word later than the text since we are trying to predict the next words. the labels are flatten into a 1d array.

In [47]:
next(iter(md.trn_dl))

(Variable containing:
     12   1414     71  ...    1478     38     48
    259      4      5  ...   25375     46      2
    398     24  30853  ...       3      2    925
         ...            ⋱           ...         
      5   2437     10  ...     337    151      2
      2    290      6  ...       8     80  14278
     77      8  15931  ...    7534     59     12
 [torch.cuda.LongTensor of size 75x64 (GPU 0)], Variable containing:
    259
      4
      5
   ⋮   
     10
   4457
  24458
 [torch.cuda.LongTensor of size 4800 (GPU 0)])

## Train



In [0]:
em_sz=200#size of each embedding vector
nh=500 #number of hidden activation per layer
nl = 3 #number of layers

Researchers have found that large amounts of momentum dont work well with these kinds of RNN models, so we create a versio of adam optimizerwith less momentum than its default of .9

In [0]:
opt_fn = partial(optim.Adam,betas=(.7,.99))

fastai uses a variant of the state of the art AWD LSTM Language model developed by Stephen Merity. A key feature of that model is that it provides excellent regularization through dropout. 

The other parameters (alpha, beta, clip) should not generally need tuning.

In [0]:
learner = md.get_model(opt_fn,em_sz,nh,nl,
                      dropouti=.05, dropout = .05, wdrop=.1, dropoute=.02, dropouth=.05)
learner.reg_fn = partial(seq2seq_reg,alpha=2,beta=1)
learner.clip=.3

In [54]:
learner

SequentialRNN(
  (0): RNN_Encoder(
    (encoder): Embedding(37392, 200, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(37392, 200, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(200, 500)
      )
      (1): WeightDrop(
        (module): LSTM(500, 500)
      )
      (2): WeightDrop(
        (module): LSTM(500, 200)
      )
    )
    (dropouti): LockedDropout(
    )
    (dropouths): ModuleList(
      (0): LockedDropout(
      )
      (1): LockedDropout(
      )
      (2): LockedDropout(
      )
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=200, out_features=37392)
    (dropout): LockedDropout(
    )
  )
)

In [0]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   
    0      5.117613   5.003315  
 17%|█▋        | 781/4583 [03:47<18:27,  3.43it/s, loss=5.05]

    1      4.779484   4.641753  


    2      4.662469   4.551414  


    3      4.660255   4.524417  


    4      4.565983   4.443225  


    5      4.504306   4.386119  
 50%|█████     | 2292/4583 [11:14<11:14,  3.40it/s, loss=4.46]

In [0]:
learner.save_encoder("adam1_enc")

In [0]:
!cp data/aclImdb/models/adam1_enc.h5 drive

In [0]:
learner.load_encoder("adam1_enc")

In [0]:
#learner.load_cycle('adam1_enc',2)

In [0]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=10)

epoch      trn_loss   val_loss   
    0      4.511491   4.389013  
 17%|█▋        | 782/4583 [03:49<18:37,  3.40it/s, loss=4.49]

    1      4.48086    4.374584  


    2      4.467634   4.357195  


    3      4.443721   4.333606  


    4      4.411171   4.310105  


    5      4.406945   4.291477  


    6      4.354332   4.270876  


    7      4.337243   4.256666  


    8      4.315477   4.248435  
 57%|█████▋    | 2624/4583 [12:52<09:36,  3.40it/s, loss=4.3]

In [0]:
learner.save_encoder('adam3_10_enc')
!cp data/aclImdb/models/adam3_10_enc.h5 drive

In [0]:
math.exp(4.165)

In [0]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [41]:
#uncomment to refresh google drive
!google-drive-ocamlfuse -cc

Clearing cache...done


In [0]:
!cp drive/imdb_adam3_c1_cl10_cyc_0.h5 data/aclImdb/models/


In [51]:
learner.model.named_parameters

<bound method Module.named_parameters of SequentialRNN(
  (0): RNN_Encoder(
    (encoder): Embedding(37392, 200, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(37392, 200, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(200, 500)
      )
      (1): WeightDrop(
        (module): LSTM(500, 500)
      )
      (2): WeightDrop(
        (module): LSTM(500, 200)
      )
    )
    (dropouti): LockedDropout(
    )
    (dropouths): ModuleList(
      (0): LockedDropout(
      )
      (1): LockedDropout(
      )
      (2): LockedDropout(
      )
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=200, out_features=37392)
    (dropout): LockedDropout(
    )
  )
)>

In [49]:
md.nt = 34945
learner

SequentialRNN(
  (0): RNN_Encoder(
    (encoder): Embedding(37392, 200, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(37392, 200, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(200, 500)
      )
      (1): WeightDrop(
        (module): LSTM(500, 500)
      )
      (2): WeightDrop(
        (module): LSTM(500, 200)
      )
    )
    (dropouti): LockedDropout(
    )
    (dropouths): ModuleList(
      (0): LockedDropout(
      )
      (1): LockedDropout(
      )
      (2): LockedDropout(
      )
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=200, out_features=37392)
    (dropout): LockedDropout(
    )
  )
)

In [50]:
learner.load('imdb_adam3_c1_cl10_cyc_0')

RuntimeError: ignored

In [90]:
!ls drive/*.h5

drive/imdb2x_cyc_4.h5  drive/imdb_adam3_c1_cl10_cyc_0.h5  drive/resnet_75.h5


#Train from scratch (including training the language model)


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab , data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import*
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy


#Language modeling

In [14]:
PATH=

aclImdb.tgz  aclImdb.tgz.1  aclImdb.tgz.2  data  datalab  drive
